In [2]:
import io

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from statistics import stdev
import time
import numpy as np
import matplotlib.pyplot as plt

In [9]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

from torch.autograd.variable import Variable



In [3]:
# load function for pretrained versions of word embeddings
def load_embeddings(emb_path, nmax=50000):
    vectors = []
    word2id = {}
    with io.open(emb_path, 'r', encoding='utf-8', newline='\n', errors='ignore') as f:
        next(f)
        for i, line in enumerate(f):
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            assert word not in word2id, 'word found twice'
            vectors.append(vect)
            word2id[word] = len(word2id)
            if len(word2id) == nmax:
                break
    id2word = {v: k for k, v in word2id.items()}
    embeddings = np.vstack(vectors)
    return embeddings, id2word, word2id

In [4]:
#eng_path = '/Users/louismonier/Downloads/Monolingual/wiki.en.vec' 
#fr_path = '/Users/louismonier/Downloads/Monolingual/wiki.fr.vec'

eng_path = '/Users/vince/DataProjetNLP/wiki.en.vec'
fr_path = '/Users/vince/DataProjetNLP/wiki.fr.vec'


nmax = 50000  # maximum number of word embeddings to load

# load monolingual word embeddings 
src_embeddings, src_id2word, src_word2id = load_embeddings(fr_path, nmax) # source = french 
tgt_embeddings, tgt_id2word, tgt_word2id = load_embeddings(eng_path, nmax) # target = english

In [5]:
# load ground-truth bilingual dictionaries function
def load_dic(path):
    dico_full = {}
    vectors_src=[]
    vectors_tgt = []
    with io.open(path,'r',encoding='utf_8') as f:
        for i,line in enumerate(f):
            word_src, word_tgt = line.rstrip().split(' ',1)
            if word_tgt in tgt_word2id :
                dico_full[word_src]=word_tgt
    for key in dico_full.keys() :
            vectors_src.append(src_embeddings[src_word2id[key]])
            vectors_tgt.append(tgt_embeddings[tgt_word2id[dico_full[key]]])
    X = np.vstack(vectors_src)
    Z = np.vstack (vectors_tgt)
    return dico_full,X,Z

In [10]:
# train & test bilingual dictionaries
path_train = '/Users/vince/DataProjetNLP/fr-en.0-5000.txt' 
path_test = '/Users/vince/DataProjetNLP/fr-en.5000-6500.txt'
dico_train, X_train, Z_train = load_dic(path_train)
dico_test, X_test, Z_test = load_dic(path_test) 
print(type(X_train[0]))
X_train, Z_train, X_test, Z_test = map(torch.tensor, (X_train, Z_train, X_test, Z_test)) #TRANSFORM INTO TORCH TENSORS
print(type(X_train[0]))

<class 'numpy.ndarray'>
<class 'torch.Tensor'>


In [11]:
# function to minimize 
# normalisation added
def C(W,X,Z):
    S = 0
    S = sum(np.linalg.norm(np.dot(X, W.T) - Z, axis=1)**2) 
    return S

In [12]:
print(X_train.shape[0], "training samples")
print(X_test.shape[0], "test samples")
dim = X_train.shape[1]
print("Vectors Dimension :", dim)

4971 training samples
1483 test samples
Vectors Dimension : 300


In [13]:
print(X_train.shape)
y = X_train.view(X_train.shape[1],-1)
print(y.shape)

torch.Size([4971, 300])
torch.Size([300, 4971])


In [33]:
class Discriminator(nn.Module):
    def __init__(self,d):
        super(Discriminator,self).__init__()
        self.h1 = nn.Linear(dim,2048,bias=True)
        self.h2 = nn.Linear(2048,2048,bias=True)
        self.out = nn.Linear(2048,1,bias=True)


    def forward(self,x):
        y = x.view(x.shape[0],-1) #Reshape 
        y = F.relu(self.h1(y))
        y = F.relu(self.h2(y))
        y = self.out(y)
        return y

In [38]:
class Mapping(nn.Module):
    def __init__(self,d):
        super(Mapping,self).__init__()
        self.h1 = nn.Linear(dim,2048,bias=True)
        self.h2 = nn.Linear(2048,2048,bias=True)
        self.out = nn.Linear(2048,1,bias=True)


    def forward(self,x):
        y = x.view(x.shape[0],-1) #Reshape 
        y = F.relu(self.h1(y))
        y = F.relu(self.h2(y))
        y = self.out(y)
        return y

In [ ]:
def LossDisc():
    return(-1/n)

In [ ]:
def LossMap():
    

In [39]:
discrim = Discriminator(300)
gen = Mapping(300)
optimizer = torch.optim.SGD(discrim.parameters(),lr = 0.1)
#criterion = nn.CrossEntropyLoss()

In [18]:
n_epochs = 10

for n in range(n_epochs):
    i = 0
    for x, y in train_loader:
        y_pred = model(x)
        loss = criterion(y_pred,y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        i+=1
        if i % 100 ==0 :
            print("Epoch [{}/{}], [{}/{}], batch loss = {}".format(n+1,n_epochs,i*bs,n_data_train,loss))      
            
    acc = 0    
    with torch.no_grad():
        for x, y in test_loader:
            y_pred = model(x)
            y_pred = torch.argmax(y_pred,dim=1)
            acc += (y_pred == y).sum().item()
        acc = acc/n_data_test
    print("Epoch [{}/{}], test accuracy = {}".format(n+1,n_epochs,acc))

Discriminator(
  (h1): Linear(in_features=300, out_features=2048, bias=True)
  (h2): Linear(in_features=2048, out_features=2048, bias=True)
  (out): Linear(in_features=2048, out_features=1, bias=True)
)